# Data Preparation

## Why do this step?

- The data we scraped from the UFC website is raw data. Every row contains information about a fight that took place i.e. details like how many strikes were thrown in that fight and who won the fight.
- To prepare the data for prediction, every row can contain only an accurate representation of what each fighter has done in fights up until that fight! No data that was recorded during the fight can be present in that row.
- Our Target variable is Winner. The task has to be to predict the winner from the data available of each fighter up until the fight

## Looking at the data

In [1]:
import pandas as pd
import numpy as np
import math

df = pd.read_csv('../data/total_fight_data.csv', sep=';')
fighter_details = pd.read_csv('../data/fighter_details.csv', index_col='fighter_name')

In [2]:
df.head()

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,...,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Brian Ortega,Chan Sung Jung,2,0,127 of 212,62 of 163,59%,38%,129 of 214,64 of 166,...,0 of 0,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Lukasz Bosacki,"October 17, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates",Featherweight Bout,Brian Ortega
1,Katlyn Chookagian,Jessica Andrade,0,1,19 of 53,28 of 43,35%,65%,27 of 61,62 of 79,...,6 of 6,KO/TKO,1,4:55,3 Rnd (5-5-5),Lukasz Bosacki,"October 17, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates",Women's Flyweight Bout,Jessica Andrade
2,Jimmy Crute,Modestas Bukauskas,2,0,17 of 24,7 of 17,70%,41%,18 of 25,7 of 17,...,0 of 0,KO/TKO,1,2:01,3 Rnd (5-5-5),Anders Ohlsson,"October 17, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates",Light Heavyweight Bout,Jimmy Crute
3,Claudio Silva,James Krause,0,0,64 of 214,94 of 159,29%,59%,66 of 216,96 of 162,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Lukasz Bosacki,"October 17, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates",Welterweight Bout,James Krause
4,Thomas Almeida,Jonathan Martinez,0,0,64 of 171,73 of 205,37%,35%,64 of 171,73 of 205,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Anders Ohlsson,"October 17, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates",Featherweight Bout,Jonathan Martinez


In [3]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,5792,5793,5794,5795,5796,5797,5798,5799,5800,5801
R_fighter,Brian Ortega,Katlyn Chookagian,Jimmy Crute,Claudio Silva,Thomas Almeida,Mateusz Gamrot,Gillian Robertson,Junyong Park,Jamie Mullarkey,Gadzhimurad Antigulov,...,Johnny Rhodes,Patrick Smith,Royce Gracie,Jason DeLucia,Remco Pardoel,Orlando Wiet,Frank Hamaker,Johnny Rhodes,Patrick Smith,Scott Morris
B_fighter,Chan Sung Jung,Jessica Andrade,Modestas Bukauskas,James Krause,Jonathan Martinez,Guram Kutateladze,Poliana Botelho,John Phillips,Fares Ziam,Maxim Grishin,...,Fred Ettish,Scott Morris,Minoki Ichihara,Scott Baker,Alberta Cerra Leon,Robert Lucarelli,Thaddeus Luster,David Levicki,Ray Wizard,Sean Daugherty
R_KD,2,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
B_KD,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R_SIG_STR.,127 of 212,19 of 53,17 of 24,64 of 214,64 of 171,52 of 126,57 of 98,10 of 21,20 of 72,12 of 23,...,13 of 29,13 of 17,2 of 4,3 of 5,4 of 6,8 of 12,2 of 3,11 of 17,1 of 1,1 of 1
B_SIG_STR.,62 of 163,28 of 43,7 of 17,94 of 159,73 of 205,37 of 86,22 of 43,5 of 14,47 of 89,34 of 76,...,4 of 7,0 of 0,3 of 7,0 of 2,1 of 3,2 of 6,0 of 0,4 of 5,1 of 1,0 of 4
R_SIG_STR_pct,59%,35%,70%,29%,37%,41%,58%,47%,27%,52%,...,44%,76%,50%,60%,66%,66%,66%,64%,100%,100%
B_SIG_STR_pct,38%,65%,41%,59%,35%,43%,51%,35%,52%,44%,...,57%,---,42%,0%,33%,33%,---,80%,100%,0%
R_TOTAL_STR.,129 of 214,27 of 61,18 of 25,66 of 216,64 of 171,69 of 151,104 of 161,260 of 350,29 of 84,31 of 44,...,21 of 38,19 of 25,110 of 114,20 of 25,20 of 22,11 of 15,14 of 15,74 of 86,1 of 1,2 of 2
B_TOTAL_STR.,64 of 166,62 of 79,7 of 17,96 of 162,73 of 205,43 of 93,39 of 61,13 of 25,54 of 98,66 of 126,...,7 of 11,0 of 0,12 of 16,14 of 23,9 of 11,2 of 6,0 of 0,95 of 102,2 of 2,1 of 5


In [4]:
df.describe()

,R_KD,B_KD,R_SUB_ATT,B_SUB_ATT,R_REV,B_REV,last_round
count,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000
mean,0.249569,0.180972,0.487246,0.347466,0.139607,0.134436,2.311961
std,0.521330,0.458590,0.930951,0.796639,0.425226,0.415428,1.007757
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,3.000000
max,5.000000,4.000000,10.000000,7.000000,5.000000,3.000000,5.000000


In [5]:
df.dtypes

R_fighter          object
B_fighter          object
R_KD                int64
B_KD                int64
R_SIG_STR.         object
B_SIG_STR.         object
R_SIG_STR_pct      object
B_SIG_STR_pct      object
R_TOTAL_STR.       object
B_TOTAL_STR.       object
R_TD               object
B_TD               object
R_TD_pct           object
B_TD_pct           object
R_SUB_ATT           int64
B_SUB_ATT           int64
R_REV               int64
B_REV               int64
R_CTRL             object
B_CTRL             object
R_HEAD             object
B_HEAD             object
R_BODY             object
B_BODY             object
R_LEG              object
B_LEG              object
R_DISTANCE         object
B_DISTANCE         object
R_CLINCH           object
B_CLINCH           object
R_GROUND           object
B_GROUND           object
win_by             object
last_round          int64
last_round_time    object
Format             object
Referee            object
date               object
location    

In [6]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR.', 'B_SIG_STR.',
       'R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'R_CTRL', 'B_CTRL', 'R_HEAD', 'B_HEAD', 'R_BODY',
       'B_BODY', 'R_LEG', 'B_LEG', 'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH',
       'B_CLINCH', 'R_GROUND', 'B_GROUND', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner'],
      dtype='object')

### Column definitions:

- `R_` and `B_` prefix signifies red and blue corner fighter stats respectively
- `KD` is number of knockdowns
- `SIG_STR` is no. of significant strikes 'landed of attempted'
- `SIG_STR_pct` is significant strikes percentage
- `TOTAL_STR` is total strikes 'landed of attempted'
- `TD` is no. of takedowns
- `TD_pct` is takedown percentages
- `SUB_ATT` is no. of submission attempts
- `PASS` is no. times the guard was passed?
- `REV` is the no. of reversals
- `CTRL` is the time spent with ground control
- `HEAD` is no. of significant strinks to the head 'landed of attempted'
- `BODY` is no. of significant strikes to the body 'landed of attempted'
- `CLINCH` is no. of significant strikes in the clinch 'landed of attempted'
- `GROUND` is no. of significant strikes on the ground 'landed of attempted'
- `win_by` is method of win
- `last_round` is last round of the fight (ex. if it was a KO in 1st, then this will be 1)
- `last_round_time` is when the fight ended in the last round
- `Format` is the format of the fight (3 rounds, 5 rounds etc.)
- `Referee` is the name of the Ref
- `date` is the date of the fight
- `location` is the location in which the event took place
- `Fight_type` is which weight class and whether it's a title bout or not
- `Winner` is the winner of the fight

#### Per fighter career wide stats
- `SLpM` - Significant Strikes Landed per Minute
- `Str_Acc.` - Significant Striking Accuracy
- `SApM` - Significant Strikes Absorbed per Minute
- `Str_Def` - Significant Strike Defence (the % of opponents strikes that did not land)
- `TD_Avg` - Average Takedowns Landed per 15 minutes
- `TD_Acc` - Takedown Accuracy
- `TD_Def` - Takedown Defense (the % of opponents TD attempts that did not land)
- `Sub_Avg` - Average Submissions Attempted per 15 minutes 

## Todo:

- Separate `landed of attempted` to separate columns

- Convert `Fight_type` into two separate columns, `weight_class` and `Title_fight` (True or False)

- Convert `last_round_time` to `total_time_fought` by using `last_round` and `Format`

- Convert `CTRL` to `time_in_CTRL`

- Convert percentages to fractions

- Since the data is a description of each fight, we have to convert it into a format that shows the compilation data of each fighter up until that fight. This means every row will look a lot different than it looks now.

- Create `current_win_streak`, `current_lose_streak`, `longest_win_streak`, `wins`, `losses`, `draw`

- Create fighter `height`, `reach`, `weight`, `age`

### Splitting landed of attempted to different columns

In [7]:
columns = ['R_SIG_STR.', 'B_SIG_STR.', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_HEAD', 'B_HEAD', 'R_BODY','B_BODY', 'R_LEG', 'B_LEG', 
        'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH','B_CLINCH', 'R_GROUND', 'B_GROUND']

for column in columns:
    print(f"{column} data type is: {df[column].dtype}")

R_SIG_STR. data type is: object
B_SIG_STR. data type is: object
R_TOTAL_STR. data type is: object
B_TOTAL_STR. data type is: object
R_TD data type is: object
B_TD data type is: object
R_HEAD data type is: object
B_HEAD data type is: object
R_BODY data type is: object
B_BODY data type is: object
R_LEG data type is: object
B_LEG data type is: object
R_DISTANCE data type is: object
B_DISTANCE data type is: object
R_CLINCH data type is: object
B_CLINCH data type is: object
R_GROUND data type is: object
B_GROUND data type is: object


In [8]:
attempt_suffix = '_att'
landed_suffix = '_landed'

for column in columns:
    df[column+attempt_suffix] = df[column].apply(lambda X: int(X.split('of')[1]))
    df[column+landed_suffix] = df[column].apply(lambda X: int(X.split('of')[0]))
    
df.drop(columns, axis=1, inplace=True)

In [9]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'R_CTRL', 'B_CTRL', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner', 'R_SIG_STR._att', 'R_SIG_STR._landed',
       'B_SIG_STR._att', 'B_SIG_STR._landed', 'R_TOTAL_STR._att',
       'R_TOTAL_STR._landed', 'B_TOTAL_STR._att', 'B_TOTAL_STR._landed',
       'R_TD_att', 'R_TD_landed', 'B_TD_att', 'B_TD_landed', 'R_HEAD_att',
       'R_HEAD_landed', 'B_HEAD_att', 'B_HEAD_landed', 'R_BODY_att',
       'R_BODY_landed', 'B_BODY_att', 'B_BODY_landed', 'R_LEG_att',
       'R_LEG_landed', 'B_LEG_att', 'B_LEG_landed', 'R_DISTANCE_att',
       'R_DISTANCE_landed', 'B_DISTANCE_att', 'B_DISTANCE_landed',
       'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att', 'B_CLINCH_landed',
       'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att', 'B_GROUND_landed'],
      dtype

### Replacing Winner NaNs as Draw

In [10]:
for column in df.columns:
    if df[column].isnull().sum() != 0:
        print(f"NaN values in {column} = {df[column].isnull().sum()}")

NaN values in Referee = 30
NaN values in Winner = 100


* 83 missing values in winner and 23 missing values in Referee

In [11]:
df[df['Winner'].isnull()]['win_by'].value_counts()

Overturned              40
Decision - Majority     24
Could Not Continue      15
Decision - Split        12
Decision - Unanimous     6
Other                    2
Submission               1
Name: win_by, dtype: int64

* Here, Overturned means due to drug test being positive and Could not Continue means there was an illegal blow which was not enough to be disqualified but the fighter could not continue.
* The rest are different forms of draw

* Replacing all of these with draw

In [12]:
df['Winner'].fillna('Draw', inplace=True)

### Converting percentages to fractions

In [13]:
pct_columns = ['R_SIG_STR_pct','B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct']

def pct_to_frac(X):
    if X != '---':
        return float(X.replace('%', ''))/100
    else:
        # if '---' means it's taking pct of `0 of 0`. 
        # Taking a call here to consider 0 landed of 0 attempted as 0 percentage
        return 0

for column in pct_columns:
    df[column] = df[column].apply(pct_to_frac)

In [14]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,5792,5793,5794,5795,5796,5797,5798,5799,5800,5801
R_fighter,Brian Ortega,Katlyn Chookagian,Jimmy Crute,Claudio Silva,Thomas Almeida,Mateusz Gamrot,Gillian Robertson,Junyong Park,Jamie Mullarkey,Gadzhimurad Antigulov,...,Johnny Rhodes,Patrick Smith,Royce Gracie,Jason DeLucia,Remco Pardoel,Orlando Wiet,Frank Hamaker,Johnny Rhodes,Patrick Smith,Scott Morris
B_fighter,Chan Sung Jung,Jessica Andrade,Modestas Bukauskas,James Krause,Jonathan Martinez,Guram Kutateladze,Poliana Botelho,John Phillips,Fares Ziam,Maxim Grishin,...,Fred Ettish,Scott Morris,Minoki Ichihara,Scott Baker,Alberta Cerra Leon,Robert Lucarelli,Thaddeus Luster,David Levicki,Ray Wizard,Sean Daugherty
R_KD,2,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
B_KD,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R_SIG_STR_pct,0.59,0.35,0.7,0.29,0.37,0.41,0.58,0.47,0.27,0.52,...,0.44,0.76,0.5,0.6,0.66,0.66,0.66,0.64,1,1
B_SIG_STR_pct,0.38,0.65,0.41,0.59,0.35,0.43,0.51,0.35,0.52,0.44,...,0.57,0,0.42,0,0.33,0.33,0,0.8,1,0
R_TD_pct,0.3,0,0,0.16,0,0.31,0.42,1,0.45,0.2,...,0,0,1,0,1,0,1,1,0,1
B_TD_pct,0,0.66,0,0,0,0,0,0,0.2,1,...,0,0,0,1,0,1,0,0,0,0
R_SUB_ATT,0,0,0,0,0,0,0,0,0,0,...,1,0,2,5,1,0,3,0,1,1
B_SUB_ATT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


### Creating a title_bout feature and weight_class

In [15]:
df['Fight_type'].value_counts()

Lightweight Bout                                                       1028
Welterweight Bout                                                      1002
Middleweight Bout                                                       745
Heavyweight Bout                                                        512
Featherweight Bout                                                      507
                                                                       ... 
Ultimate Fighter Brazil 3 Heavyweight Tournament Title Bout               1
Ultimate Fighter Latin America 2 Lightweight Tournament Title Bout        1
Ultimate Fighter 22 Lightweight Tournament Title Bout                     1
Ultimate Japan Heavyweight Tournament Title Bout                          1
TUF Nations Canada vs. Australia Middleweight Tournament Title Bout       1
Name: Fight_type, Length: 109, dtype: int64

In [16]:
df['Fight_type'].value_counts()[df['Fight_type'].value_counts() > 1].index

Index(['Lightweight Bout', 'Welterweight Bout', 'Middleweight Bout',
       'Heavyweight Bout', 'Featherweight Bout', 'Light Heavyweight Bout',
       'Bantamweight Bout', 'Flyweight Bout', 'Women's Strawweight Bout',
       'Women's Bantamweight Bout', 'Women's Flyweight Bout',
       'Open Weight Bout', 'UFC Light Heavyweight Title Bout',
       'UFC Welterweight Title Bout', 'UFC Heavyweight Title Bout',
       'Catch Weight Bout', 'UFC Middleweight Title Bout',
       'UFC Lightweight Title Bout', 'UFC Flyweight Title Bout',
       'UFC Women's Bantamweight Title Bout', 'UFC Bantamweight Title Bout',
       'UFC Featherweight Title Bout', 'UFC Women's Strawweight Title Bout',
       'Women's Featherweight Bout', 'UFC Women's Featherweight Title Bout',
       'UFC Interim Heavyweight Title Bout',
       'UFC Superfight Championship Bout', 'UFC Women's Flyweight Title Bout',
       'UFC Interim Featherweight Title Bout',
       'UFC Interim Middleweight Title Bout',
       'UFC Inter

In [17]:
df['title_bout'] = df['Fight_type'].apply(lambda X: True if 'Title Bout' in X else False)

In [18]:
def make_weight_class(X):
    for weight_class in weight_classes:
        if weight_class in X:
            return weight_class
    if X == 'Catch Weight Bout' or 'Catchweight Bout':
        return 'Catch Weight'
    else:
        return 'Open Weight'

In [19]:
weight_classes = ['Women\'s Strawweight', 'Women\'s Bantamweight', 
                  'Women\'s Featherweight', 'Women\'s Flyweight', 'Lightweight', 
                  'Welterweight', 'Middleweight','Light Heavyweight', 
                  'Heavyweight', 'Featherweight','Bantamweight', 'Flyweight', 'Open Weight']

df['weight_class'] = df['Fight_type'].apply(make_weight_class)

In [20]:
df[df['weight_class'].isnull()]['Fight_type'].value_counts()

Series([], Name: Fight_type, dtype: int64)

### Creating total_time_fought

In [21]:
df['Format'].value_counts()

3 Rnd (5-5-5)           5101
5 Rnd (5-5-5-5-5)        485
1 Rnd + OT (12-3)         80
No Time Limit             29
3 Rnd + OT (5-5-5-5)      22
1 Rnd (20)                21
1 Rnd + 2OT (15-3-3)      20
2 Rnd (5-5)               14
1 Rnd (15)                 8
1 Rnd (10)                 6
1 Rnd (12)                 4
1 Rnd + OT (30-5)          3
1 Rnd (18)                 2
1 Rnd + OT (15-3)          2
1 Rnd + OT (27-3)          1
1 Rnd + OT (31-5)          1
1 Rnd + OT (30-3)          1
1 Rnd (30)                 1
1 Rnd + 2OT (24-3-3)       1
Name: Format, dtype: int64

In [22]:
df['Format'].value_counts().index

Index(['3 Rnd (5-5-5)', '5 Rnd (5-5-5-5-5)', '1 Rnd + OT (12-3)',
       'No Time Limit', '3 Rnd + OT (5-5-5-5)', '1 Rnd (20)',
       '1 Rnd + 2OT (15-3-3)', '2 Rnd (5-5)', '1 Rnd (15)', '1 Rnd (10)',
       '1 Rnd (12)', '1 Rnd + OT (30-5)', '1 Rnd (18)', '1 Rnd + OT (15-3)',
       '1 Rnd + OT (27-3)', '1 Rnd + OT (31-5)', '1 Rnd + OT (30-3)',
       '1 Rnd (30)', '1 Rnd + 2OT (24-3-3)'],
      dtype='object')

In [23]:
time_in_first_round = {'3 Rnd (5-5-5)': 5*60, '5 Rnd (5-5-5-5-5)': 5*60, '1 Rnd + OT (12-3)': 12*60,
       'No Time Limit': 1, '3 Rnd + OT (5-5-5-5)': 5*60, '1 Rnd (20)': 1*20,
       '2 Rnd (5-5)': 5*60, '1 Rnd (15)': 15*60, '1 Rnd (10)': 10*60,
       '1 Rnd (12)':12*60, '1 Rnd + OT (30-5)': 30*60, '1 Rnd (18)': 18*60, '1 Rnd + OT (15-3)': 15*60,
       '1 Rnd (30)': 30*60, '1 Rnd + OT (31-5)': 31*5,
       '1 Rnd + OT (27-3)': 27*60, '1 Rnd + OT (30-3)': 30*60}

exception_format_time = {'1 Rnd + 2OT (15-3-3)': [15*60, 3*60], '1 Rnd + 2OT (24-3-3)': [24*60, 3*60]}

# '1 Rnd + 2OT (15-3-3)' and '1 Rnd + 2OT (24-3-3)' is not included because it has 3 uneven timed rounds. 
# We'll have to deal with it separately

In [24]:
# Converting to seconds
df['last_round_time'] = df['last_round_time'].apply(lambda X: int(X.split(':')[0])*60 + int(X.split(':')[1]))

In [25]:
def get_total_time(row):
    if row['Format'] in time_in_first_round.keys():
        return (row['last_round'] - 1) * time_in_first_round[row['Format']] + row['last_round_time']
    elif row['Format'] in exception_format_time.keys():
        if (row['last_round'] - 1) >= 2:
            return exception_format_time[row['Format']][0] + (row['last_round'] - 2) * \
                    exception_format_time[row['Format']][1] + row['last_round_time']
        else:
            return (row['last_round'] - 1) * exception_format_time[row['Format']][0] + row['last_round_time']
    
# So if the fight ended in round 1, we only need last_round_time. 
# If it ended in round 2, we need the full time of round 1 and the last_round_time
# This works for fights with same time in each round and fights with only two rounds.

In [26]:
df['total_time_fought(seconds)'] = df.apply(get_total_time, axis=1)

In [27]:
def get_no_of_rounds(X):
    if X == 'No Time Limit':
        return 1
    else:
        return len(X.split('(')[1].replace(')', '').split('-'))

df['no_of_rounds'] = df['Format'].apply(get_no_of_rounds)

In [28]:
df

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR_pct,B_SIG_STR_pct,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,...,B_CLINCH_att,B_CLINCH_landed,R_GROUND_att,R_GROUND_landed,B_GROUND_att,B_GROUND_landed,title_bout,weight_class,total_time_fought(seconds),no_of_rounds
0,Brian Ortega,Chan Sung Jung,2,0,0.59,0.38,0.30,0.00,0,0,...,1,0,6,4,0,0,False,Featherweight,1500,5
1,Katlyn Chookagian,Jessica Andrade,0,1,0.35,0.65,0.00,0.66,0,0,...,9,8,0,0,6,6,False,Women's Flyweight,295,3
2,Jimmy Crute,Modestas Bukauskas,2,0,0.70,0.41,0.00,0.00,0,0,...,0,0,2,2,0,0,False,Light Heavyweight,121,3
3,Claudio Silva,James Krause,0,0,0.29,0.59,0.16,0.00,0,0,...,0,0,0,0,0,0,False,Welterweight,900,3
4,Thomas Almeida,Jonathan Martinez,0,0,0.37,0.35,0.00,0.00,0,0,...,4,4,0,0,0,0,False,Featherweight,900,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5797,Orlando Wiet,Robert Lucarelli,0,0,0.66,0.33,0.00,1.00,0,1,...,0,0,9,7,0,0,False,Open Weight,170,1
5798,Frank Hamaker,Thaddeus Luster,0,0,0.66,0.00,1.00,0.00,3,0,...,0,0,2,1,0,0,False,Open Weight,292,1
5799,Johnny Rhodes,David Levicki,0,0,0.64,0.80,1.00,0.00,0,0,...,2,2,15,9,1,1,False,Open Weight,733,1
5800,Patrick Smith,Ray Wizard,0,0,1.00,1.00,0.00,0.00,1,0,...,0,0,0,0,0,0,False,Open Weight,58,1


In [29]:
df.drop(['Format', 'Fight_type', 'last_round_time'], axis = 1, inplace=True)

### Create CTRL_time(seconds)

In [30]:
CTRL_columns = ['R_CTRL','B_CTRL']

def conv_to_sec(X):
    if X != '--':
        return int(X.split(':')[0])*60 + int(X.split(':')[1])
    else:
        # if '--' means there was no time spent on the ground. 
        # Taking a call here to consider this as 0 seconds
        return 0

for column in CTRL_columns:
    df[column+'_time(seconds)'] = df[column].apply(conv_to_sec)

In [31]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,5792,5793,5794,5795,5796,5797,5798,5799,5800,5801
R_fighter,Brian Ortega,Katlyn Chookagian,Jimmy Crute,Claudio Silva,Thomas Almeida,Mateusz Gamrot,Gillian Robertson,Junyong Park,Jamie Mullarkey,Gadzhimurad Antigulov,...,Johnny Rhodes,Patrick Smith,Royce Gracie,Jason DeLucia,Remco Pardoel,Orlando Wiet,Frank Hamaker,Johnny Rhodes,Patrick Smith,Scott Morris
B_fighter,Chan Sung Jung,Jessica Andrade,Modestas Bukauskas,James Krause,Jonathan Martinez,Guram Kutateladze,Poliana Botelho,John Phillips,Fares Ziam,Maxim Grishin,...,Fred Ettish,Scott Morris,Minoki Ichihara,Scott Baker,Alberta Cerra Leon,Robert Lucarelli,Thaddeus Luster,David Levicki,Ray Wizard,Sean Daugherty
R_KD,2,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
B_KD,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R_SIG_STR_pct,0.59,0.35,0.7,0.29,0.37,0.41,0.58,0.47,0.27,0.52,...,0.44,0.76,0.5,0.6,0.66,0.66,0.66,0.64,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
weight_class,Featherweight,Women's Flyweight,Light Heavyweight,Welterweight,Featherweight,Lightweight,Women's Flyweight,Middleweight,Lightweight,Light Heavyweight,...,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight
total_time_fought(seconds),1500,295,121,900,900,900,900,900,900,598,...,187,30,308,401,591,170,292,733,58,20
no_of_rounds,5,3,3,3,3,3,3,3,3,3,...,1,1,1,1,1,1,1,1,1,1
R_CTRL_time(seconds),57,25,25,58,2,209,574,791,416,159,...,0,0,0,0,0,0,0,0,0,0


In [32]:
df.drop(['R_CTRL','B_CTRL'], axis = 1, inplace=True)

### Create another DataFrame to save the compiled data per fighter (Our Prediction DataFrame)

In [33]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'win_by', 'last_round', 'Referee', 'date', 'location',
       'Winner', 'R_SIG_STR._att', 'R_SIG_STR._landed', 'B_SIG_STR._att',
       'B_SIG_STR._landed', 'R_TOTAL_STR._att', 'R_TOTAL_STR._landed',
       'B_TOTAL_STR._att', 'B_TOTAL_STR._landed', 'R_TD_att', 'R_TD_landed',
       'B_TD_att', 'B_TD_landed', 'R_HEAD_att', 'R_HEAD_landed', 'B_HEAD_att',
       'B_HEAD_landed', 'R_BODY_att', 'R_BODY_landed', 'B_BODY_att',
       'B_BODY_landed', 'R_LEG_att', 'R_LEG_landed', 'B_LEG_att',
       'B_LEG_landed', 'R_DISTANCE_att', 'R_DISTANCE_landed', 'B_DISTANCE_att',
       'B_DISTANCE_landed', 'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att',
       'B_CLINCH_landed', 'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att',
       'B_GROUND_landed', 'title_bout', 'weight_class',
       'total_time_fought(seconds)', 'no_of_rounds',

In [34]:
df2 = df.copy()

In [35]:
df2.drop(['R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_CTRL_time(seconds)', 'B_CTRL_time(seconds)', 'R_REV', 'B_REV', 'win_by', 'last_round', 
        'R_SIG_STR._att', 'R_SIG_STR._landed',
       'B_SIG_STR._att', 'B_SIG_STR._landed', 'R_TOTAL_STR._att',
       'R_TOTAL_STR._landed', 'B_TOTAL_STR._att', 'B_TOTAL_STR._landed',
       'R_TD_att', 'R_TD_landed', 'B_TD_att', 'B_TD_landed', 'R_HEAD_att',
       'R_HEAD_landed', 'B_HEAD_att', 'B_HEAD_landed', 'R_BODY_att',
       'R_BODY_landed', 'B_BODY_att', 'B_BODY_landed', 'R_LEG_att',
       'R_LEG_landed', 'B_LEG_att', 'B_LEG_landed', 'R_DISTANCE_att',
       'R_DISTANCE_landed', 'B_DISTANCE_att', 'B_DISTANCE_landed',
       'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att', 'B_CLINCH_landed',
       'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att', 'B_GROUND_landed',
        'total_time_fought(seconds)'], axis = 1, inplace=True)
df2

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds
0,Brian Ortega,Chan Sung Jung,Lukasz Bosacki,"October 17, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates",Brian Ortega,False,Featherweight,5
1,Katlyn Chookagian,Jessica Andrade,Lukasz Bosacki,"October 17, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates",Jessica Andrade,False,Women's Flyweight,3
2,Jimmy Crute,Modestas Bukauskas,Anders Ohlsson,"October 17, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates",Jimmy Crute,False,Light Heavyweight,3
3,Claudio Silva,James Krause,Lukasz Bosacki,"October 17, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates",James Krause,False,Welterweight,3
4,Thomas Almeida,Jonathan Martinez,Anders Ohlsson,"October 17, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates",Jonathan Martinez,False,Featherweight,3
...,...,...,...,...,...,...,...,...,...
5797,Orlando Wiet,Robert Lucarelli,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Orlando Wiet,False,Open Weight,1
5798,Frank Hamaker,Thaddeus Luster,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Frank Hamaker,False,Open Weight,1
5799,Johnny Rhodes,David Levicki,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Johnny Rhodes,False,Open Weight,1
5800,Patrick Smith,Ray Wizard,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Patrick Smith,False,Open Weight,1


### Compiling Data per fighter

In [36]:
red_fighters = df['R_fighter'].value_counts().index
blue_fighters = df['B_fighter'].value_counts().index

fighters = list(set(red_fighters) | set(blue_fighters))

In [37]:
def get_renamed_winner(row):
    if row['R_fighter'] == row['Winner']:
        return 'Red'
    elif row['B_fighter'] == row['Winner']:
        return 'Blue'
    elif row['Winner'] == 'Draw':
        return 'Draw'

df2['Winner'] = df2[['R_fighter', 'B_fighter', 'Winner']].apply(get_renamed_winner, axis=1)

In [38]:
df = pd.concat([df,pd.get_dummies(df['win_by'], prefix='win_by')],axis=1)
df.drop(['win_by'],axis=1, inplace=True)

In [39]:
Numerical_columns = ['hero_KD', 'opp_KD', 'hero_SIG_STR_pct',
       'opp_SIG_STR_pct', 'hero_TD_pct', 'opp_TD_pct', 'hero_SUB_ATT', 'opp_SUB_ATT',
        'hero_REV', 'opp_REV', 'hero_SIG_STR._att', 'hero_SIG_STR._landed',
       'opp_SIG_STR._att', 'opp_SIG_STR._landed', 'hero_TOTAL_STR._att',
       'hero_TOTAL_STR._landed', 'opp_TOTAL_STR._att', 'opp_TOTAL_STR._landed',
       'hero_TD_att', 'hero_TD_landed', 'opp_TD_att', 'opp_TD_landed', 'hero_HEAD_att',
       'hero_HEAD_landed', 'opp_HEAD_att', 'opp_HEAD_landed', 'hero_BODY_att',
       'hero_BODY_landed', 'opp_BODY_att', 'opp_BODY_landed', 'hero_LEG_att',
       'hero_LEG_landed', 'opp_LEG_att', 'opp_LEG_landed', 'hero_DISTANCE_att',
       'hero_DISTANCE_landed', 'opp_DISTANCE_att', 'opp_DISTANCE_landed',
       'hero_CLINCH_att', 'hero_CLINCH_landed', 'opp_CLINCH_att', 'opp_CLINCH_landed',
       'hero_GROUND_att', 'hero_GROUND_landed', 'opp_GROUND_att', 'opp_GROUND_landed', 
        'hero_CTRL_time(seconds)', 'opp_CTRL_time(seconds)',
       'total_time_fought(seconds)']

Categorical_columns = ['win_by', 'last_round',
        'Winner', 'title_bout']

For all `Numerical_columns`, we take the average of those columns for every fighter of every fight they had up until that point.

For `Categorical_columns`, we have to come up with different ideas for each column:

* Each `win_by` will be a column of it's own
* from `last_round` we can get, `total_rounds_fought`
* from `total_time_fought` we can get `average_time_fought`
* from `Winner` we get `wins`, `losses`, `draw`, `current_streak`, `longest_streak`
* from `title_bout` we can get `no_of_title_fights`

In [40]:
import re

def lreplace(pattern, sub, string):
    """
    Replaces 'pattern' in 'string' with 'sub' if 'pattern' starts 'string'.
    """
    return re.sub('^%s' % pattern, sub, string)

In [41]:
red = df.groupby('R_fighter')
blue = df.groupby('B_fighter')

In [42]:
def get_fighter_red(fighter_name):
    try:
        fighter_red = red.get_group(fighter_name)
    except:
        return None
    rename_columns = {}
    for column in fighter_red.columns:
        if re.search('^R_', column) is not None:
            rename_columns[column] = lreplace('R_', 'hero_', column)
        elif re.search('^B_', column) is not None:
            rename_columns[column] = lreplace('B_', 'opp_', column)
    fighter_red = fighter_red.rename(rename_columns, axis='columns')
    return fighter_red

In [43]:
def get_fighter_blue(fighter_name):
    try:
        fighter_blue = blue.get_group(fighter_name)
    except:
        return None
    rename_columns = {}
    for column in fighter_blue.columns:
        if re.search('^B_', column) is not None:
            rename_columns[column] = lreplace('B_', 'hero_', column)
        elif re.search('^R_', column) is not None:
            rename_columns[column] = lreplace('R_', 'opp_', column)
    fighter_blue = fighter_blue.rename(rename_columns, axis='columns')
    return fighter_blue

In [44]:
def get_result_stats(result_list):
    result_list.reverse() # To get it in ascending order
    current_win_streak = 0
    current_lose_streak = 0
    longest_win_streak = 0
    wins = 0
    losses = 0
    draw = 0
    for result in result_list:
        if result == 'hero':
            wins += 1
            current_win_streak += 1
            current_lose_streak = 0
            if longest_win_streak < current_win_streak:
                longest_win_streak += 1
        elif result == 'opp':
            losses += 1
            current_win_streak = 0
            current_lose_streak += 1
        elif result == 'draw':
            draw += 1
            current_lose_streak = 0
            current_win_streak = 0
            
    return current_win_streak, current_lose_streak, longest_win_streak, wins, losses, draw

In [45]:
win_by_columns = ['win_by_Decision - Majority', 'win_by_Decision - Split',
       'win_by_Decision - Unanimous', 'win_by_KO/TKO','win_by_Submission',
       'win_by_TKO - Doctor\'s Stoppage']

In [46]:
temp_blue_frame = pd.DataFrame()
temp_red_frame = pd.DataFrame()
result_stats = ['current_win_streak', 'current_lose_streak', 'longest_win_streak', 'wins', 'losses', 'draw']

for fighter_name in fighters:
    fighter_red = get_fighter_red(fighter_name)
    fighter_blue = get_fighter_blue(fighter_name)
    fighter_index = None
    
    if fighter_red is None:
        fighter = fighter_blue
        fighter_index = 'blue'
    elif fighter_blue is None:
        fighter = fighter_red
        fighter_index = 'red'
    else:
        fighter = pd.concat([fighter_red, fighter_blue]).sort_index(ascending=False)
    
    fighter['Winner'] = fighter['Winner'].apply(lambda X: 'hero' if X == fighter_name else 'opp')

    for i, index in enumerate(fighter.index):
        fighter_slice = fighter[(i+1):].sort_index(ascending=False)
        s = fighter_slice[Numerical_columns].ewm(span=3, adjust=False).mean().tail(1)
        if len(s) != 0:
            pass
        else:
            s.loc[len(s)] = [np.NaN for _ in s.columns]
        s['total_rounds_fought'] = fighter_slice['last_round'].sum()
        s['total_title_bouts'] = fighter_slice[fighter_slice['title_bout']==True]['title_bout'].count()
        s['hero_fighter'] = fighter_name
        results = get_result_stats(list(fighter_slice['Winner']))
        for result_stat, result in zip(result_stats, results):
            s[result_stat] = result
        win_by_results = fighter_slice[fighter_slice['Winner'] == 'hero'][win_by_columns].sum()
        for win_by_column,win_by_result in zip(win_by_columns, win_by_results):
            s[win_by_column] = win_by_result
        s.index = [index]


        if fighter_index is None:
            if index in fighter_blue.index:
                temp_blue_frame = temp_blue_frame.append(s)
            elif index in fighter_red.index:
                temp_red_frame = temp_red_frame.append(s)
        elif fighter_index == 'blue':
            temp_blue_frame = temp_blue_frame.append(s)
        elif fighter_index == 'red':
            temp_red_frame = temp_red_frame.append(s)

In [47]:
temp_blue_frame.T

,1130,3629,3726,1702,710,473,1165,788,330,156,...,5046,4877,4820,4644,317,4569,4241,4126,3059,2812
hero_KD,0,0,NaN,0.625,0.5,1,0.09375,0.125,0,NaN,...,0.0625,0,0,NaN,NaN,0.015625,0,0,0,NaN
opp_KD,0,0,NaN,0,0,0,0.25,0.25,0.5,NaN,...,0.5,0.5,1,NaN,NaN,0.046875,0.0625,0.0625,0,NaN
hero_SIG_STR_pct,0.4375,0.29,NaN,0.423125,0.355,0.5,0.439375,0.44,0.43,NaN,...,0.33625,0.32,0.26,NaN,NaN,0.396484,0.3925,0.40125,0.38,NaN
opp_SIG_STR_pct,0.4225,0.51,NaN,0.438125,0.41,0.41,0.425938,0.41125,0.475,NaN,...,0.39625,0.425,0.36,NaN,NaN,0.53709,0.537187,0.53875,0.67,NaN
hero_TD_pct,0.125,0.16,NaN,0.0575,0,0,0.185,0.15625,0.125,NaN,...,0.143125,0.155,0.2,NaN,NaN,0.382656,0.37875,0.37875,0.33,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
win_by_Decision - Split,0,0,0,0,0,0,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0
win_by_Decision - Unanimous,1,0,0,2,0,0,2,1,0,0,...,0,0,0,0,0,2,1,1,0,0
win_by_KO/TKO,0,0,0,1,1,1,0,0,0,0,...,1,0,0,0,0,3,0,0,0,0
win_by_Submission,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,1,1,0,0


### Adding fighter details like height, weight, reach, stance and dob

In [48]:
fighter_details = fighter_details[fighter_details.index.isin(fighters)]
for col in fighter_details.columns:
    print(f"Number of NaN in {col} : {fighter_details[col].isnull().sum()}")

Number of NaN in Height : 13
Number of NaN in Weight : 10
Number of NaN in Reach : 645
Number of NaN in Stance : 75
Number of NaN in DOB : 141
Number of NaN in SLpM : 0
Number of NaN in Str_Acc : 0
Number of NaN in SApM : 0
Number of NaN in Str_Def : 0
Number of NaN in TD_Avg : 0
Number of NaN in TD_Acc : 0
Number of NaN in TD_Def : 0
Number of NaN in Sub_Avg : 0


In [49]:
def convert_to_cms(X):
    if X is np.NaN:
        return X
    elif len(X.split("'")) == 2:
        feet = float(X.split("'")[0])
        inches = int(X.split("'")[1].replace(' ', '').replace('"',''))
        return (feet * 30.48) + (inches * 2.54)
    else:
        return float(X.replace('"','')) * 2.54

In [50]:
fighter_details['Height_cms'] = fighter_details['Height'].apply(convert_to_cms)
fighter_details['Reach_cms'] = fighter_details['Reach'].apply(convert_to_cms)

In [51]:
fighter_details['Weight_lbs'] = fighter_details['Weight'].apply(lambda X: float(X.replace(' lbs.', '')) if X is not np.NaN else X)

In [52]:
pct_columns = ['Str_Acc','Str_Def', 'TD_Acc', 'TD_Def']

def pct_to_frac(X):
    if X != np.NaN:
        return float(X.replace('%', ''))/100
    else:
        return 0

for column in pct_columns:
    fighter_details[column] = fighter_details[column].apply(pct_to_frac)

In [53]:
fighter_details.drop(['Height', 'Weight', 'Reach'], axis=1, inplace=True)

In [54]:
fighter_details.reset_index(inplace=True)
temp_red_frame.reset_index(inplace=True)
temp_blue_frame.reset_index(inplace=True)

In [55]:
temp_blue_frame = temp_blue_frame.merge(fighter_details, left_on='hero_fighter', right_on='fighter_name', how='left')
temp_blue_frame.set_index('index', inplace=True)

In [56]:
temp_blue_frame[['hero_fighter', 'fighter_name', 'Height_cms', 'Weight_lbs', 'DOB']].head(20)

,hero_fighter,fighter_name,Height_cms,Weight_lbs,DOB
index,,,,,
1130,Brad Katona,Brad Katona,167.64,135.0,"Dec 19, 1991"
3629,Motonobu Tezuka,Motonobu Tezuka,170.18,135.0,"Aug 29, 1987"
3726,Motonobu Tezuka,Motonobu Tezuka,170.18,135.0,"Aug 29, 1987"
1702,Andrew Sanchez,Andrew Sanchez,185.42,185.0,"Apr 08, 1988"
710,Andrew Sanchez,Andrew Sanchez,185.42,185.0,"Apr 08, 1988"
473,Andrew Sanchez,Andrew Sanchez,185.42,185.0,"Apr 08, 1988"
1165,Dan Ige,Dan Ige,170.18,145.0,"Aug 06, 1991"
788,Dan Ige,Dan Ige,170.18,145.0,"Aug 06, 1991"
330,Dan Ige,Dan Ige,170.18,145.0,"Aug 06, 1991"


In [57]:
temp_red_frame = temp_red_frame.merge(fighter_details, left_on='hero_fighter', right_on='fighter_name', how='left')
temp_red_frame.set_index('index', inplace=True)

In [58]:
temp_blue_frame.drop('fighter_name', axis=1, inplace=True)
temp_red_frame.drop('fighter_name', axis=1, inplace=True)

In [59]:
blue_frame = temp_blue_frame.add_prefix('B_')
red_frame = temp_red_frame.add_prefix('R_')

In [60]:
frame = blue_frame.join(red_frame, how='outer')

In [61]:
rename_cols = {}
for col in frame.columns:
    if 'hero' in col:
        rename_cols[col] = col.replace('_hero_', '_avg_').replace('.', '')
    if 'opp' in col:
        rename_cols[col] = col.replace('_opp_', '_avg_opp_').replace('.', '')
    if 'win_by' in col:
        rename_cols[col] = col.replace(' ', '').replace('-', '_').replace('\'s', '_')

In [62]:
frame.rename(rename_cols, axis='columns', inplace=True)

In [63]:
frame.drop(['R_avg_fighter','B_avg_fighter'], axis=1, inplace=True)

In [64]:
df2 = df2.join(frame, how='outer')

### Create Age

In [65]:
df2['R_DOB'] = pd.to_datetime(df2['R_DOB'])
df2['B_DOB'] = pd.to_datetime(df2['B_DOB'])
df2['date'] = pd.to_datetime(df2['date'])

In [66]:
def get_age(row):
    B_age = (row['date'] - row['B_DOB']).days
    R_age = (row['date'] - row['R_DOB']).days
    if np.isnan(B_age)!=True:
        B_age = math.floor(B_age/365.25)
    if np.isnan(R_age)!=True:
        R_age = math.floor(R_age/365.25)
    return pd.Series([B_age, R_age], index=['B_age', 'R_age'])

In [67]:
df2[['B_age', 'R_age']]= df2[['date', 'R_DOB', 'B_DOB']].apply(get_age, axis=1)

In [68]:
df2.drop(['R_DOB', 'B_DOB'], axis=1, inplace=True)

In [69]:
# df2.drop(df2.index[df2['Winner'] == 'draw'], inplace = True)

In [70]:
# df2.to_csv('../data/data.csv', index=False)

In [71]:
df2

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_avg_KD,...,R_Str_Def,R_TD_Avg,R_TD_Acc,R_TD_Def,R_Sub_Avg,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Brian Ortega,Chan Sung Jung,Lukasz Bosacki,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Red,False,Featherweight,5,NaN,...,0.52,0.80,0.21,0.56,1.1,172.72,175.26,145.0,33.0,29.0
1,Katlyn Chookagian,Jessica Andrade,Lukasz Bosacki,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Blue,False,Women's Flyweight,3,NaN,...,0.62,0.30,0.15,0.48,0.5,175.26,172.72,125.0,29.0,31.0
2,Jimmy Crute,Modestas Bukauskas,Anders Ohlsson,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Red,False,Light Heavyweight,3,NaN,...,0.54,4.33,0.75,0.60,2.4,187.96,187.96,205.0,26.0,24.0
3,Claudio Silva,James Krause,Lukasz Bosacki,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Blue,False,Welterweight,3,NaN,...,0.43,3.01,0.28,0.66,1.6,180.34,180.34,170.0,34.0,38.0
4,Thomas Almeida,Jonathan Martinez,Anders Ohlsson,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Blue,False,Featherweight,3,1.625,...,0.64,0.00,0.00,0.75,0.0,170.18,177.80,145.0,26.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5797,Orlando Wiet,Robert Lucarelli,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0.00,0.00,0.00,0.00,0.0,177.80,NaN,170.0,NaN,NaN
5798,Frank Hamaker,Thaddeus Luster,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0.00,0.00,0.00,0.00,0.0,NaN,NaN,NaN,NaN,NaN
5799,Johnny Rhodes,David Levicki,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0.00,0.00,0.00,0.00,0.0,182.88,NaN,210.0,NaN,NaN
5800,Patrick Smith,Ray Wizard,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0.00,0.00,0.00,0.00,0.0,187.96,NaN,225.0,NaN,30.0
